In [1]:
from pyspark.sql import SparkSession
import numpy as np

### Se tiene un problema de registros recorridos con valores en columnas que no les corresponden

Vi que creando tu propio schema forzas a que los valores sean de cierto tipo en específico y cuando pase el error de este dataset

lo va a transformar a null. Ese problema se resuelve después eliminando los registros con valores null.

### Creción de un schema

In [3]:
################################################################################################
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

mi_esquema = StructType([
    StructField('FECHA_ACTUALIZACION', DateType(), True),
    StructField('ID_REGISTRO', StringType(), True),
    StructField('ORIGEN', StringType(), True),
    StructField('SECTOR', StringType(), True),
    StructField('ENTIDAD_UM', IntegerType(), True),
    StructField('SEXO', IntegerType(), True),
    StructField('ENTIDAD_NAC', IntegerType(), True),
    StructField('ENTIDAD_RES', IntegerType(), True),
    StructField('MUNICIPIO_RES', IntegerType(), True),
    StructField('FECHA_INGRESO', DateType(), True),
    StructField('FECHA_SINTOMAS', DateType(), True),
    StructField('FECHA_DEF', DateType(), True),
    StructField('INTUBADO', IntegerType(), True),
    StructField('NEUMONIA', IntegerType(), True),
    StructField('EDAD', IntegerType(), True),
    StructField('NACIONALIDAD', IntegerType(), True),
    StructField('EMBARAZO', IntegerType(), True),
    StructField('HABLA_LENGUA_INDIG', IntegerType(), True),
    StructField('INDIGENA', IntegerType(), True),
    StructField('DIABETES', IntegerType(), True),
    StructField('ASMA', IntegerType(), True),
    StructField('INMUSUPR', IntegerType(), True),
    StructField('HIPERTENSION', IntegerType(), True),
    StructField('OTRA_COM', IntegerType(), True),
    StructField('CARDIOVASCULAR', IntegerType(), True),
    StructField('OBESIDAD', IntegerType(), True),
    StructField('RENAL_CRONICA', IntegerType(), True),
    StructField('OTRO_CASO', IntegerType(), True),
    StructField('TOMA_MUESTRA_LAB', IntegerType(), True),
    StructField('RESULTADO_LAB', IntegerType(), True),
    StructField('TOMA_MUESTRA_ANTIGENO', IntegerType(), True),
    StructField('RESULTADO_ANTIGENO', IntegerType(), True),
    StructField('CLASIFICACION_FINAL', IntegerType(), True),
    StructField('MIGRANTE', IntegerType(), True),
    StructField('PAIS_NACIONALIDAD', StringType(), True),
    StructField('PAIS_ORIGEN', StringType(), True),
    StructField('UCI', IntegerType(), True)])


In [3]:
spark = SparkSession.builder.appName('embarazos').getOrCreate()

ruta = 'DatosCovid/Covid_limpio.txt'
df = spark.read.csv(
    ruta, 
    header = True,
    inferSchema=True,
    mode = 'PERMISSIVE',
    multiLine = False,
    escape = "\""
)

df.show()

25/12/06 15:04:19 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------------------------------------+----------+-----+-----+------+-----+-----+------+------+----+-------------+-------------+-------------+-----+----+-----+----+----+-------+-----+---------+----+-------------+-------------+-------------+---------+--------+------+------+
|1 16734179 11 faltan datos ['2022-07-20'| 'g18107d'| '2'2|  '4'| '15'4| '2'5| '09'| '15'7|   '02|  29|          210|          211|          212|  213| 214|  215| 216| 217|    218|  219|      220|  97|            1|          223|            7|       99|México"'|  '97'| '97']|
+----------------------------------------+----------+-----+-----+------+-----+-----+------+------+----+-------------+-------------+-------------+-----+----+-----+----+----+-------+-----+---------+----+-------------+-------------+-------------+---------+--------+------+------+
|                    1 FECHA_ACTUALIZA...|      NULL| NULL| NULL|  NULL| NULL| NULL|  NULL|  NULL|NULL|         NULL|         NULL|         NULL| NULL|NULL| NULL|NULL|NU

25/12/06 15:04:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 1 16734179 11 faltan datos ['2022-07-20',  'g18107d',  '2',  '4',  '15',  '2',  '09',  '15',  '02, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 97, 1, 2, 7, 99, México"',  '97',  '97']
 Schema: 1 16734179 11 faltan datos ['2022-07-20',  'g18107d',  '2'2,  '4',  '15'4,  '2'5,  '09',  '15'7,  '02, 29, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 97, 1, 223, 7, 99, México"',  '97',  '97']
Expected:  '2'2 but found:  '2'
CSV file: file:///home/jair/Escritorio/Big%20Data/Proyectos/Covid/DatosCovid/Covid_limpio.txt


### ELiminar unas columnas de basura

In [5]:
fechas = [col for col in df.columns if 'FECHA_' in col]
eliminar = ['ID_REGISTRO'] + fechas
df = df.drop(*eliminar)
df.show()

+------+------+----------+----+-----------+-----------+-------------+--------+--------+----+------------+--------+------------------+--------+--------+----+--------+------------+--------+--------------+--------+-------------+---------+----------------+-------------+---------------------+------------------+-------------------+--------+-----------------+-----------+---+
|ORIGEN|SECTOR|ENTIDAD_UM|SEXO|ENTIDAD_NAC|ENTIDAD_RES|MUNICIPIO_RES|INTUBADO|NEUMONIA|EDAD|NACIONALIDAD|EMBARAZO|HABLA_LENGUA_INDIG|INDIGENA|DIABETES|ASMA|INMUSUPR|HIPERTENSION|OTRA_COM|CARDIOVASCULAR|OBESIDAD|RENAL_CRONICA|OTRO_CASO|TOMA_MUESTRA_LAB|RESULTADO_LAB|TOMA_MUESTRA_ANTIGENO|RESULTADO_ANTIGENO|CLASIFICACION_FINAL|MIGRANTE|PAIS_NACIONALIDAD|PAIS_ORIGEN|UCI|
+------+------+----------+----+-----------+-----------+-------------+--------+--------+----+------------+--------+------------------+--------+--------+----+--------+------------+--------+--------------+--------+-------------+---------+----------------+------

25/12/06 14:01:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ORIGEN, SECTOR, ENTIDAD_UM, SEXO, ENTIDAD_NAC, ENTIDAD_RES, MUNICIPIO_RES, FECHA_DEF, INTUBADO, NEUMONIA, EDAD, NACIONALIDAD, EMBARAZO, HABLA_LENGUA_INDIG, INDIGENA, DIABETES, EPOC, ASMA, INMUSUPR, HIPERTENSION, OTRA_COM, CARDIOVASCULAR, OBESIDAD, RENAL_CRONICA, TABAQUISMO, OTRO_CASO, TOMA_MUESTRA_LAB, RESULTADO_LAB, TOMA_MUESTRA_ANTIGENO, RESULTADO_ANTIGENO, CLASIFICACION_FINAL, MIGRANTE
 Schema: ORIGEN, SECTOR, ENTIDAD_UM, SEXO, ENTIDAD_NAC, ENTIDAD_RES, MUNICIPIO_RES, INTUBADO, NEUMONIA, EDAD, NACIONALIDAD, EMBARAZO, HABLA_LENGUA_INDIG, INDIGENA, DIABETES, ASMA, INMUSUPR, HIPERTENSION, OTRA_COM, CARDIOVASCULAR, OBESIDAD, RENAL_CRONICA, OTRO_CASO, TOMA_MUESTRA_LAB, RESULTADO_LAB, TOMA_MUESTRA_ANTIGENO, RESULTADO_ANTIGENO, CLASIFICACION_FINAL, MIGRANTE, PAIS_NACIONALIDAD, PAIS_ORIGEN, UCI
Expected: INTUBADO but found: FECHA_DEF
CSV file: file:///home/jair/Escritorio/Big%20Data/Data/DatosCovid/

In [6]:
df.dtypes

[('ORIGEN', 'string'),
 ('SECTOR', 'string'),
 ('ENTIDAD_UM', 'int'),
 ('SEXO', 'int'),
 ('ENTIDAD_NAC', 'int'),
 ('ENTIDAD_RES', 'int'),
 ('MUNICIPIO_RES', 'int'),
 ('INTUBADO', 'int'),
 ('NEUMONIA', 'int'),
 ('EDAD', 'int'),
 ('NACIONALIDAD', 'int'),
 ('EMBARAZO', 'int'),
 ('HABLA_LENGUA_INDIG', 'int'),
 ('INDIGENA', 'int'),
 ('DIABETES', 'int'),
 ('ASMA', 'int'),
 ('INMUSUPR', 'int'),
 ('HIPERTENSION', 'int'),
 ('OTRA_COM', 'int'),
 ('CARDIOVASCULAR', 'int'),
 ('OBESIDAD', 'int'),
 ('RENAL_CRONICA', 'int'),
 ('OTRO_CASO', 'int'),
 ('TOMA_MUESTRA_LAB', 'int'),
 ('RESULTADO_LAB', 'int'),
 ('TOMA_MUESTRA_ANTIGENO', 'int'),
 ('RESULTADO_ANTIGENO', 'int'),
 ('CLASIFICACION_FINAL', 'int'),
 ('MIGRANTE', 'int'),
 ('PAIS_NACIONALIDAD', 'string'),
 ('PAIS_ORIGEN', 'string'),
 ('UCI', 'int')]

# Verificar los casos con menores registros, ahi me puedo encontrar errores

In [7]:
for columna in df.columns:
    result = df.groupBy(df[columna]).count().orderBy('count', ascending = True).collect()
    print(f'Valores de la característica {columna} ------------------------------------------------------------')
    print('Conteo   |   Valor')
    for row in result:
        print(f'{row[1]} - {row[0]}')

# Si me aparecieron varios errores de dedo que son la minoría, entonces si mas adelante voy a usar una característica
# Tengo que hacer una limpieza de esos pocos valores que no son representativos

Valores de la característica ORIGEN ------------------------------------------------------------
Conteo   |   Valor
3 - 97
3 - 9999-99-99
4051118 - 1
13259918 - 2


Valores de la característica SECTOR ------------------------------------------------------------
Conteo   |   Valor
1 - 34
5 - 97
36 - 14
820 - 99
2764 - 2
3351 - 1
8725 - 13
13215 - 7
24022 - 11
44248 - 10
74640 - 8
80424 - 5
174596 - 3
414022 - 6
1015964 - 9
6059893 - 4
9394316 - 12


Valores de la característica ENTIDAD_UM ------------------------------------------------------------
Conteo   |   Valor
1 - 73
2 - None
108336 - 4
123251 - 29
124317 - 6
133258 - 18
157384 - 32
179392 - 10
200875 - 13
205768 - 7
211514 - 1
212780 - 23
231943 - 20
235236 - 12
262461 - 8
291259 - 31
299275 - 3
318891 - 22
323476 - 16
328351 - 17
364365 - 26
369889 - 28
370077 - 2
373808 - 25
382839 - 30
420859 - 5
426193 - 21
564813 - 14
587496 - 24
587511 - 27
739181 - 11
771478 - 19
1055151 - 15
6349612 - 9


Valores de la característica SEXO ------------------------------------------------------------
Conteo   |   Valor
1 - 52
1 - 43
1 - 97
1 - 36
2 - None
8019223 - 2
9291813 - 1


Valores de la característica ENTIDAD_NAC ------------------------------------------------------------
Conteo   |   Valor
3 - None
104285 - 6
104810 - 4
114785 - 23
124821 - 99
134893 - 29
136830 - 18
174000 - 32
194007 - 3
201556 - 10
201674 - 1
220749 - 2
224417 - 22
230168 - 13
250021 - 8
269957 - 7
288379 - 17
305227 - 31
322767 - 20
343736 - 12
353288 - 26
360270 - 28
397946 - 16
426986 - 5
428922 - 25
488672 - 21
558768 - 24
579399 - 14
583309 - 30
590412 - 27
662123 - 19
744218 - 11
1824337 - 15
5365307 - 9


...

25/12/06 14:07:22 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: CLASIFICACION_FINAL
 Schema: PAIS_ORIGEN
Expected: PAIS_ORIGEN but found: CLASIFICACION_FINAL
CSV file: file:///home/jair/Escritorio/Big%20Data/Data/DatosCovid/220720COVID19MEXICO.csv


Valores de la característica PAIS_NACIONALIDAD ------------------------------------------------------------
Conteo   |   Valor
1 - 5
1 - 9999-99-99
1 - 014
1 - 2022-01-12
12 - None
3941478 - 1
5613637 - 97
7755911 - 2


Valores de la característica PAIS_ORIGEN ------------------------------------------------------------
Conteo   |   Valor
1 - 9999-99-99
2 - 97
12 - None
11726 - 4
15792 - 2
91461 - 5
351208 - 1
640559 - 6
6156001 - 3
10044280 - 7


25/12/06 14:07:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: MIGRANTE
 Schema: UCI
Expected: UCI but found: MIGRANTE
CSV file: file:///home/jair/Escritorio/Big%20Data/Data/DatosCovid/220720COVID19MEXICO.csv


Valores de la característica UCI ------------------------------------------------------------
Conteo   |   Valor
2 - 97
13 - None
23924 - 1
90199 - 2
17196904 - 99


### Registros por país

In [24]:
result = df.groupBy(df['PAIS_NACIONALIDAD']).count().orderBy('count', ascending = False).collect()
for row in result:
    print(f'{row[1]} - {row[0]}')

25/10/24 19:00:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: RESULTADO_ANTIGENO
 Schema: PAIS_NACIONALIDAD
Expected: PAIS_NACIONALIDAD but found: RESULTADO_ANTIGENO
CSV file: file:///home/jair/Escritorio/Big%20Data/Data/DatosCovid/220720COVID19MEXICO.csv


7755911 - 2
5613637 - 97
3941478 - 1
12 - None
1 - 5
1 - 9999-99-99
1 - 014
1 - 2022-01-12


In [25]:
antes_null = df.count()

### Eliminarle los nulos al dataset

In [26]:
df = df.na.drop()
despues_null = df.count()
print(f'Datos nulos eliminados: {antes_null - despues_null}')

25/10/24 19:01:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ORIGEN, SECTOR, ENTIDAD_UM, SEXO, ENTIDAD_NAC, ENTIDAD_RES, MUNICIPIO_RES, FECHA_DEF, INTUBADO, NEUMONIA, EDAD, NACIONALIDAD, EMBARAZO, HABLA_LENGUA_INDIG, INDIGENA, DIABETES, EPOC, ASMA, INMUSUPR, HIPERTENSION, OTRA_COM, CARDIOVASCULAR, OBESIDAD, RENAL_CRONICA, TABAQUISMO, OTRO_CASO, TOMA_MUESTRA_LAB, RESULTADO_LAB, TOMA_MUESTRA_ANTIGENO, RESULTADO_ANTIGENO, CLASIFICACION_FINAL, MIGRANTE
 Schema: ORIGEN, SECTOR, ENTIDAD_UM, SEXO, ENTIDAD_NAC, ENTIDAD_RES, MUNICIPIO_RES, INTUBADO, NEUMONIA, EDAD, NACIONALIDAD, EMBARAZO, HABLA_LENGUA_INDIG, INDIGENA, DIABETES, ASMA, INMUSUPR, HIPERTENSION, OTRA_COM, CARDIOVASCULAR, OBESIDAD, RENAL_CRONICA, OTRO_CASO, TOMA_MUESTRA_LAB, RESULTADO_LAB, TOMA_MUESTRA_ANTIGENO, RESULTADO_ANTIGENO, CLASIFICACION_FINAL, MIGRANTE, PAIS_NACIONALIDAD, PAIS_ORIGEN, UCI
Expected: INTUBADO but found: FECHA_DEF
CSV file: file:///home/jair/Escritorio/Big%20Data/Data/DatosCovid/

Datos nulos eliminados: 17311042


### ELIMINAR LO QUE NO QUIERO USAR

In [27]:
fechas = [col for col in df.columns if 'FECHA_' in col]
eliminar = ['ID_REGISTRO', 'INTUBADO'] + fechas
df = df.drop(*eliminar)
df.show()

25/10/24 19:02:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ORIGEN, SECTOR, ENTIDAD_UM, SEXO, ENTIDAD_NAC, ENTIDAD_RES, MUNICIPIO_RES, FECHA_DEF, INTUBADO, NEUMONIA, EDAD, NACIONALIDAD, EMBARAZO, HABLA_LENGUA_INDIG, INDIGENA, DIABETES, EPOC, ASMA, INMUSUPR, HIPERTENSION, OTRA_COM, CARDIOVASCULAR, OBESIDAD, RENAL_CRONICA, TABAQUISMO, OTRO_CASO, TOMA_MUESTRA_LAB, RESULTADO_LAB, TOMA_MUESTRA_ANTIGENO, RESULTADO_ANTIGENO, CLASIFICACION_FINAL, MIGRANTE
 Schema: ORIGEN, SECTOR, ENTIDAD_UM, SEXO, ENTIDAD_NAC, ENTIDAD_RES, MUNICIPIO_RES, INTUBADO, NEUMONIA, EDAD, NACIONALIDAD, EMBARAZO, HABLA_LENGUA_INDIG, INDIGENA, DIABETES, ASMA, INMUSUPR, HIPERTENSION, OTRA_COM, CARDIOVASCULAR, OBESIDAD, RENAL_CRONICA, OTRO_CASO, TOMA_MUESTRA_LAB, RESULTADO_LAB, TOMA_MUESTRA_ANTIGENO, RESULTADO_ANTIGENO, CLASIFICACION_FINAL, MIGRANTE, PAIS_NACIONALIDAD, PAIS_ORIGEN, UCI
Expected: INTUBADO but found: FECHA_DEF
CSV file: file:///home/jair/Escritorio/Big%20Data/Data/DatosCovid/

+------+------+----------+----+-----------+-----------+-------------+--------+----+------------+--------+------------------+--------+--------+----+--------+------------+--------+--------------+--------+-------------+---------+----------------+-------------+---------------------+------------------+-------------------+--------+-----------------+-----------+---+
|ORIGEN|SECTOR|ENTIDAD_UM|SEXO|ENTIDAD_NAC|ENTIDAD_RES|MUNICIPIO_RES|NEUMONIA|EDAD|NACIONALIDAD|EMBARAZO|HABLA_LENGUA_INDIG|INDIGENA|DIABETES|ASMA|INMUSUPR|HIPERTENSION|OTRA_COM|CARDIOVASCULAR|OBESIDAD|RENAL_CRONICA|OTRO_CASO|TOMA_MUESTRA_LAB|RESULTADO_LAB|TOMA_MUESTRA_ANTIGENO|RESULTADO_ANTIGENO|CLASIFICACION_FINAL|MIGRANTE|PAIS_NACIONALIDAD|PAIS_ORIGEN|UCI|
+------+------+----------+----+-----------+-----------+-------------+--------+----+------------+--------+------------------+--------+--------+----+--------+------------+--------+--------------+--------+-------------+---------+----------------+-------------+-------------------

### Cambiarle los datos a numéricos para trabajarlos

In [11]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

numericas = [col for col in df.columns if col != 'PAIS_NACIONALIDAD']
for c in numericas:
    df = df.withColumn(c, col(c).cast(IntegerType()))

df.dtypes

[('ORIGEN', 'int'),
 ('SECTOR', 'int'),
 ('ENTIDAD_UM', 'int'),
 ('SEXO', 'int'),
 ('ENTIDAD_NAC', 'int'),
 ('ENTIDAD_RES', 'int'),
 ('MUNICIPIO_RES', 'int'),
 ('TIPO_PACIENTE', 'int'),
 ('NEUMONIA', 'int'),
 ('EDAD', 'int'),
 ('NACIONALIDAD', 'int'),
 ('EMBARAZO', 'int'),
 ('HABLA_LENGUA_INDIG', 'int'),
 ('INDIGENA', 'int'),
 ('DIABETES', 'int'),
 ('EPOC', 'int'),
 ('ASMA', 'int'),
 ('INMUSUPR', 'int'),
 ('HIPERTENSION', 'int'),
 ('OTRA_COM', 'int'),
 ('CARDIOVASCULAR', 'int'),
 ('OBESIDAD', 'int'),
 ('RENAL_CRONICA', 'int'),
 ('TABAQUISMO', 'int'),
 ('OTRO_CASO', 'int'),
 ('TOMA_MUESTRA_LAB', 'int'),
 ('RESULTADO_LAB', 'int'),
 ('TOMA_MUESTRA_ANTIGENO', 'int'),
 ('RESULTADO_ANTIGENO', 'int'),
 ('CLASIFICACION_FINAL', 'int'),
 ('MIGRANTE', 'int'),
 ('PAIS_NACIONALIDAD', 'string'),
 ('PAIS_ORIGEN', 'int'),
 ('UCI', 'int')]

## Experimentar con los groupBy

Quiero saber:

* Cuántas mujeres hay por cada país
* Cuántas mujeres hay en total en comparación con los hombres
* Cuántas mujeres se infectaron y cuantas no
* Cuántas mujeres estaban embarazadas y cuántas no
* Cuantas de mujeres están embarazadas estaban contagiadas

In [12]:
'''
Cuántas mujeres hay de cada país
'''
n_mujeres = df.filter(df['SEXO'] == 1).count()
print(f'Hay un total de {n_mujeres} mujeres')
df.filter(df['SEXO'] == 1).groupBy('PAIS_NACIONALIDAD').count().orderBy('count', ascending = False).show()

Hay un total de 9291807 mujeres


+--------------------+-------+
|   PAIS_NACIONALIDAD|  count|
+--------------------+-------+
|              México|9234466|
|Estados Unidos de...|  25705|
|           Venezuela|   5036|
|            Colombia|   4019|
|República de Hond...|   1989|
|           Guatemala|   1804|
|                Cuba|   1433|
|              Brasil|   1252|
|         El Salvador|   1248|
|           Argentina|   1243|
|           Nicaragua|   1036|
|              España|   1034|
|                Otro|    875|
|                Perú|    866|
|             Ecuador|    819|
|               Haití|    807|
|              Canadá|    798|
|             Francia|    790|
|            Alemania|    691|
|               Chile|    558|
+--------------------+-------+
only showing top 20 rows


In [13]:
'''
Cuántas mujeres hay en total en comparación con los hombres
'''
total = df.count()
n_mujeres = df.filter(df['SEXO'] == 1).count()
n_hombres = df.filter(df['SEXO'] == 2).count()

prop_m = n_mujeres / total
prop_h = n_hombres / total

print(f'La proporción de mujeres: {np.round(prop_m * 100, 2)} %')
print(f'La proporción de hombres: {np.round(prop_h * 100, 2)} %')

La proporción de mujeres: 53.68 %
La proporción de hombres: 46.32 %


In [28]:
from pyspark.sql.functions import col, expr

# 1. Convertir a bigint usando try_cast
df = df.withColumn("CLASIFICACION_FINAL_INT", expr("try_cast(CLASIFICACION_FINAL as bigint)"))

# 2. Filtrar solo valores válidos y materializar
df_validos = df.filter(col("CLASIFICACION_FINAL_INT").isNotNull()).cache()

# 3. Contar mujeres con clasificación positiva usando la columna casteada
positivos_covid = [1, 2, 3]
mujeres_positivas = df_validos.filter(
    (col('SEXO') == 1) & (col('CLASIFICACION_FINAL_INT').isin(positivos_covid))).count()

print(mujeres_positivas)


25/10/24 19:03:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ORIGEN, SECTOR, ENTIDAD_UM, SEXO, ENTIDAD_NAC, ENTIDAD_RES, MUNICIPIO_RES, FECHA_DEF, INTUBADO, NEUMONIA, EDAD, NACIONALIDAD, EMBARAZO, HABLA_LENGUA_INDIG, INDIGENA, DIABETES, EPOC, ASMA, INMUSUPR, HIPERTENSION, OTRA_COM, CARDIOVASCULAR, OBESIDAD, RENAL_CRONICA, TABAQUISMO, OTRO_CASO, TOMA_MUESTRA_LAB, RESULTADO_LAB, TOMA_MUESTRA_ANTIGENO, RESULTADO_ANTIGENO, CLASIFICACION_FINAL, MIGRANTE
 Schema: ORIGEN, SECTOR, ENTIDAD_UM, SEXO, ENTIDAD_NAC, ENTIDAD_RES, MUNICIPIO_RES, INTUBADO, NEUMONIA, EDAD, NACIONALIDAD, EMBARAZO, HABLA_LENGUA_INDIG, INDIGENA, DIABETES, ASMA, INMUSUPR, HIPERTENSION, OTRA_COM, CARDIOVASCULAR, OBESIDAD, RENAL_CRONICA, OTRO_CASO, TOMA_MUESTRA_LAB, RESULTADO_LAB, TOMA_MUESTRA_ANTIGENO, RESULTADO_ANTIGENO, CLASIFICACION_FINAL, MIGRANTE, PAIS_NACIONALIDAD, PAIS_ORIGEN, UCI
Expected: INTUBADO but found: FECHA_DEF
CSV file: file:///home/jair/Escritorio/Big%20Data/Data/DatosCovid/

0
